
Sequence-to-Sequence Modeling with nn.Transformer and TorchText
===============================================================

![](https://github.com/pytorch/tutorials/blob/gh-pages/_static/img/transformer_architecture.jpg?raw=1)





In [1]:
!pip uninstall -y torch torchtext
!pip install torch==2.0.0 torchtext==0.15.1

Found existing installation: torch 2.4.0
Uninstalling torch-2.4.0:
  Successfully uninstalled torch-2.4.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 5.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 86.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 71.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 3.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 4.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━

In [2]:
import torch
import torch.nn as nn
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim
import time

import os
import math

# Defina o comprimento máximo das sequências
MAX_LENGTH = 100

# Defina o tokenizador
tokenizer = get_tokenizer('basic_english')

In [3]:
# Verificar se a GPU está disponível
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Exibir a informação
if device.type == 'cuda':
    print("Rodando na GPU:", torch.cuda.get_device_name(0))
else:
    print("Rodando na CPU")


Rodando na GPU: Tesla P100-PCIE-16GB


In [4]:
# Função para ler os dados de texto
def load_wikitext(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return f.read().splitlines()

In [5]:
# Caminhos para os arquivos no Google Drive
folder = '/kaggle/input/wikitext-2'
train_file = os.path.join(folder, 'wiki.train.tokens')
val_file = os.path.join(folder, 'wiki.valid.tokens')
test_file = os.path.join(folder, 'wiki.test.tokens')

In [6]:
# Carregar os dados
train_txt = load_wikitext(train_file)
val_txt = load_wikitext(val_file)
test_txt = load_wikitext(test_file)

In [7]:
# Função para truncar ou preencher tokens
def truncate_or_pad(tokens, max_length):
    if len(tokens) > max_length:
        return tokens[:max_length]
    else:
        return tokens + ['<pad>'] * (max_length - len(tokens))

In [8]:
# Tokenizar e ajustar as sequências
train_tokens = [truncate_or_pad(tokenizer(line), MAX_LENGTH) for line in train_txt]
val_tokens = [truncate_or_pad(tokenizer(line), MAX_LENGTH) for line in val_txt]
test_tokens = [truncate_or_pad(tokenizer(line), MAX_LENGTH) for line in test_txt]

In [9]:
# Defina os tokens especiais
specials = ['<unk>', '<sos>', '<eos>', '<pad>']

In [10]:
# Função para construir o vocabulário com tokens especiais
def build_vocab_with_specials(tokens_list, specials):
    # Adicione tokens especiais ao vocabulário
    vocab = build_vocab_from_iterator(tokens_list, specials=specials, min_freq=1)
    vocab.set_default_index(vocab["<unk>"])
    return vocab

In [11]:
# Criar o vocabulário com tokens especiais
vocab = build_vocab_with_specials(train_tokens, specials)

In [12]:
# Função para converter tokens em índices usando o vocabulário
def tokenize_and_numericalize(data, vocab):
    # Converta tokens em índices inteiros e garanta que o tensor seja do tipo Long
    return [torch.tensor([vocab[token] for token in tokenizer(line)], dtype=torch.long) for line in data]

In [13]:
# Numeralizar os dados de treino, validação e teste
train_data = tokenize_and_numericalize(train_txt, vocab)
val_data = tokenize_and_numericalize(val_txt, vocab)
test_data = tokenize_and_numericalize(test_txt, vocab)

In [14]:
# Definir parâmetros do modelo
ntokens = len(vocab)  # O tamanho do vocabulário
emsize = 200          # Dimensão do embedding
nhid = 200            # Dimensão do feedforward network model
nlayers = 2           # Número de camadas do TransformerEncoder
nhead = 2             # Número de cabeças na multiheadattention
dropout = 0.2         # Valor de dropout
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
# Definir modelo Transformer
class TransformerModel(nn.Module):
    def __init__(self, ntokens, emsize, nhead, nhid, nlayers, dropout=0.2):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.encoder = nn.Embedding(ntokens, emsize)
        self.pos_encoder = nn.Sequential(
            nn.Dropout(dropout)
        )
        encoder_layers = nn.TransformerEncoderLayer(emsize, nhead, nhid, dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, nlayers)
        self.decoder = nn.Linear(emsize, ntokens)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        src = self.encoder(src) * math.sqrt(emsize)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        output = self.decoder(output)
        return output

In [16]:
# Inicializar o modelo
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)

In [17]:
# Função para dividir os dados em batches
def batchify(data, bsz):
    # Converte tokens em tensores e preenche com zeros
    data = pad_sequence([item for item in data], batch_first=True)
    nbatch = data.size(0) // bsz
    data = data[:nbatch * bsz].view(bsz, -1).t().contiguous()
    return data.to(device, dtype=torch.long)

batch_size = 20
train_data = batchify(train_data, batch_size)
val_data = batchify(val_data, batch_size)
test_data = batchify(test_data, batch_size)

def get_batch(source, i, bptt):
    seq_len = min(bptt, len(source) - i - 1)
    data = source[i:i + seq_len]
    target = source[i + 1:i + 1 + seq_len]

    if len(data) < bptt:
        padding_len = bptt - len(data)
        data = torch.cat([data, torch.full((padding_len, data.size(1)), vocab['<pad>'], dtype=torch.long)], dim=0)
    if len(target) < bptt:
        padding_len = bptt - len(target)
        # Correctly pad the target tensor along dimension 0
        target = torch.cat([target, torch.full((padding_len,), vocab['<pad>'], dtype=torch.long).unsqueeze(0).repeat(target.size(1), 1).t()], dim=0)
    return data, target

In [21]:
# Ajuste a função get_batch para garantir que os tensores estejam no mesmo dispositivo
def get_batch(source, i, bptt, device):
    data = source[i:i + bptt]
    target = source[i + 1:i + 1 + bptt]  # O alvo é deslocado em uma posição
    
    # Se a sequência de dados for menor que o bptt, adicione padding
    if len(data) < bptt:
        padding_len = bptt - len(data)
        # Criar o padding e mover para o mesmo dispositivo do 'data'
        padding = torch.full((padding_len, data.size(1)), vocab['<pad>'], dtype=torch.long).to(device)
        data = torch.cat([data, padding], dim=0)

    if len(target) < bptt:
        padding_len = bptt - len(target)
        # Criar o padding e mover para o mesmo dispositivo do 'target'
        padding = torch.full((padding_len, target.size(1)), vocab['<pad>'], dtype=torch.long).to(device)
        target = torch.cat([target, padding], dim=0)
    
    return data.to(device), target.to(device)

In [21]:
# Configurações de treinamento
bptt = 35  # Exemplo de comprimento do batch de sequência
criterion = nn.CrossEntropyLoss()

# Definir o dispositivo (GPU ou CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Pegue uma amostra para checar
src = train_data[:batch_size]
data, target = get_batch(src, 0, bptt, device)  # Exemplo de dados e alvos

# Chamar a função com o dispositivo correto
check_and_adjust_batch(data, target, model, criterion, device)

Data shape: torch.Size([35, 20])
Target shape: torch.Size([35, 20])
Model output shape: torch.Size([35, 20, 28378])
Adjusted output shape: torch.Size([700, 28378])
Adjusted target shape: torch.Size([700])
Loss: 10.11091423034668


In [22]:
# Inicializar o modelo
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)

In [30]:
# Função de treinamento
def train():
    model.train()  # Ativar o modo de treinamento
    total_loss = 0.
    start_time = time.time()
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i, bptt,device)
        optimizer.zero_grad()
        output = model(data)
        output_flat = output.view(-1, ntokens)
        loss = criterion(output_flat, targets.view(-1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:.5f} | loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_data) // bptt,
                lr, cur_loss, math.exp(cur_loss)))
            total_loss = 0.
            start_time = time.time()  # Reiniciar o tempo

In [31]:
# Função de avaliação
def evaluate(eval_model, data_source):
    eval_model.eval()  # Ativar o modo de avaliação
    total_loss = 0.
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i, bptt,device)
            output = eval_model(data)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets.view(-1)).item()
    return total_loss / (len(data_source) - 1)

In [32]:
# Configurações de treinamento
n_epochs = 5
lr = 5.0
batch_size = 20
bptt = 35
log_interval = 200

In [33]:
# Inicializar o otimizador
optimizer = optim.SGD(model.parameters(), lr=lr)

In [34]:
# Inicializar o scheduler (opcional)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [35]:
# Treinamento e avaliação
start_time = time.time()  # Iniciar a contagem do tempo

In [36]:
%%time
for epoch in range(1, n_epochs + 1):
    train()
    val_loss = evaluate(model, val_data)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | valid ppl {:8.2f}'.format(
        epoch, time.time() - start_time, val_loss, math.exp(val_loss)))
    print('-' * 89)
    scheduler.step()  # Atualizar a taxa de aprendizado

| epoch   1 |   200/36490 batches | lr 5.00000 | loss   nan | ppl      nan
| epoch   1 |   400/36490 batches | lr 5.00000 | loss   nan | ppl      nan
| epoch   1 |   600/36490 batches | lr 5.00000 | loss   nan | ppl      nan
| epoch   1 |   800/36490 batches | lr 5.00000 | loss   nan | ppl      nan
| epoch   1 |  1000/36490 batches | lr 5.00000 | loss   nan | ppl      nan
| epoch   1 |  1200/36490 batches | lr 5.00000 | loss   nan | ppl      nan
| epoch   1 |  1400/36490 batches | lr 5.00000 | loss   nan | ppl      nan
| epoch   1 |  1600/36490 batches | lr 5.00000 | loss   nan | ppl      nan
| epoch   1 |  1800/36490 batches | lr 5.00000 | loss   nan | ppl      nan
| epoch   1 |  2000/36490 batches | lr 5.00000 | loss   nan | ppl      nan
| epoch   1 |  2200/36490 batches | lr 5.00000 | loss   nan | ppl      nan
| epoch   1 |  2400/36490 batches | lr 5.00000 | loss   nan | ppl      nan
| epoch   1 |  2600/36490 batches | lr 5.00000 | loss   nan | ppl      nan
| epoch   1 |  2800/36490